In [ ]:
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

from fake_useragent import UserAgent

import nltk
nltk.download("punkt")

In [94]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [106]:
# Open webpage
driver.get("https://www.publico.pt/prova-dos-factos")

In [107]:
# After rejecting the cookies and maximizing the window, run this cell to load all news
more_button_check = True
while(more_button_check):   
    # Scroll down until the end of the webpage
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    # match=False
    # while(match==False):
    #         lastCount = lenOfPage
    #         time.sleep(3)
    #         lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    #         if lastCount==lenOfPage:
    #             match=True
    
    time.sleep(1)

    driver.execute_script("window.scrollBy(0, -100);")

    time.sleep(1)

    # Press button to load more news articles
    button_element = driver.find_element_by_class_name("stack__footer")
    try:
        driver.find_element_by_class_name("stack__footer").click()
    except:
        more_button_check = False

In [108]:
urls= set() #284
soup = bs4(driver.page_source, 'html.parser')
card_elements = soup.find_all("li", class_ = "stack__slice__item")
for element in card_elements:
    h3_tags = element.select('h3.card__title.headline, h3.card__title.headline_')
    for tag in h3_tags:
        href = tag.find("a")["href"]
        urls.add(href)

In [109]:
len(urls) #284

266

In [110]:
for url in urls:
   print(url)

https://www.publico.pt/2023/01/10/politica/noticia/imagem-lula-silva-algemado-partilhada-andre-ventura-verdadeira-2034525
https://www.publico.pt/2022/12/09/sociedade/noticia/sns-actualmente-medicos-especialistas-final-2015-2030840
https://www.publico.pt/2022/08/06/p3/noticia/estrela-chourico-cientista-frances-admite-partilhou-imagem-falsa-twitter-2016378
https://www.publico.pt/2021/12/28/politica/noticia/rui-rio-propos-acordo-parlamentar-dois-anos-ps-argumenta-proprio-1990117
https://www.publico.pt/2023/03/03/politica/noticia/verdade-nao-marcelo-anunciar-discurso-lula-sessao-25-abril-2041026
https://www.publico.pt/2022/07/18/desporto/noticia/carro-cristiano-ronaldo-recentemente-estacionado-alvalade-2014101
https://www.publico.pt/2023/01/15/mundo/noticia/trolls-propaganda-frio-russia-quer-minar-opiniao-publica-inverno-2034635
https://www.publico.pt/2022/02/24/mundo/noticia/video-paraquedistas-russos-aterrarem-kharkiv-ucrania-recente-1996685
https://www.publico.pt/2021/09/24/politica/not

In [111]:
# Remove from urls
urls_to_remove = [
    "https://www.publico.pt/2022/07/14/azul/noticia/mito-beber-leite-protegenos-fumo-incendios-2013680",
    "https://www.publico.pt/2023/01/15/mundo/noticia/trolls-propaganda-frio-russia-quer-minar-opiniao-publica-inverno-2034635",
    "https://www.publico.pt/2019/09/23/politica/noticia/catarina-martins-mostrou-abertura-viabilizar-governo-ps-eleicoes-2015-1887664",
    "https://www.publico.pt/2023/04/04/impar/noticia/politica-francesa-marlene-schiappa-posou-nua-lingerie-playboy-2044934?ref=prova-dos-factos&cx=stack",
    "https://www.publico.pt/2023/02/15/sociedade/noticia/80-media-portugueses-ja-difundiu-noticias-falsas-baseadas-redes-sociais-2039014",
    "https://www.publico.pt/2022/09/23/mundo/noticia/bolsonaro-lula-usam-dados-incorretos-trocam-farpas-recta-final-eleicoes-brasil-2021581",
    "https://www.publico.pt/2023/02/09/sociedade/noticia/policia-judiciaria-alerta-mensagem-falsa-parceria-mbway-2038255",
    "https://www.publico.pt/2019/09/18/politica/noticia/fim-corrupcao-colocaria-portugal-nivel-finlandia-1887101",
    "https://www.publico.pt/2019/09/18/sociedade/noticia/prova-factos-jornalistas-podem-condenados-violar-segredo-justica-1887122",
    "https://www.publico.pt/2022/11/29/tecnologia/noticia/twitter-reverte-politica-combater-desinformacao-covid19-2029648",
    "https://www.publico.pt/2022/11/21/impar/noticia/the-crown-familia-real-britanica-abandonou-primos-russos-1917-2028521",
    "https://www.publico.pt/2023/02/11/opiniao/opiniao/prova-factos-distorceu-factos-2038452",
    "https://www.publico.pt/2022/08/06/p3/noticia/estrela-chourico-cientista-frances-admite-partilhou-imagem-falsa-twitter-2016378",
    "https://www.publico.pt/2023/05/03/newsletter/prova-dos-factos",
    "https://www.publico.pt/2023/06/30/tecnologia/noticia/publico-recebe-financiamento-google-combater-desinformacao-2055217",
    "https://www.publico.pt/2021/05/24/tecnologia/noticia/sistema-antimisseis-combater-desinformacao-europa-partir-peninsula-iberica-1963847",
    "https://www.publico.pt/2022/05/14/azul/noticia/duvida-comer-alimentos-fim-prazo-validade-2006133",
    "https://www.publico.pt/2023/06/19/tecnologia/noticia/aviso-pagina-fraudulenta-imita-site-publico-2053807",
    "https://www.publico.pt/2023/06/05/mundo/entrevista/luke-obrien-extremadireita-ja-usa-inteligencia-artificial-desinformar-radicalizar-2052056",
    "https://www.publico.pt/2022/10/14/mundo/noticia/eleicoes-brasil-esquerda-direita-travam-guerra-narrativas-segunda-volta-2024031",
    "https://www.publico.pt/2021/01/29/ciencia/noticia/variante-reino-unido-potencial-maior-letalidade-ministra-saude-1948543",
    "https://www.publico.pt/2021/02/06/politica/noticia/governo-pediu-ajuda-internacional-combater-covid19-1949559",
    "https://www.publico.pt/2021/02/13/politica/noticia/prova-factos-possivel-governo-unidade-nacional-1950526",
    "https://www.publico.pt/2022/03/07/economia/noticia/governo-ganhar-aumento-preco-combustiveis-1997871",
    "https://www.publico.pt/2022/10/07/mundo/noticia/apos-eleicoes-eleitores-bolsonaro-espalham-falsas-acusacoes-fraude-redes-sociais-2023127",
    "https://www.publico.pt/2023/04/07/politica/noticia/recibos-verdes-estado-aumentaram-14-2045319",
    "https://www.publico.pt/2023/04/04/impar/noticia/politica-francesa-marlene-schiappa-posou-nua-lingerie-playboy-2044934",
    "https://www.publico.pt/2021/06/11/economia/noticia/ana-paula-vitorino-ira-salario-maior-pedro-adao-silva-rui-rio-1966168",
    "https://www.publico.pt/2021/09/13/desporto/noticia/treinador-dinamo-kiev-mircea-lucescu-treinador-titulos-ganhos-actividade-1977311",
    "https://www.publico.pt/2022/07/13/azul/noticia/unico-responsavel-incendios-accao-humana-antonio-costa-2013363",
    "https://www.publico.pt/2023/06/01/azul/noticia/grupo-internacional-activistas-esvazia-pneus-carros-suv-chegou-portugal-2051821",
    "https://www.publico.pt/2022/02/04/politica/noticia/candidaturas-vicepresidentes-pe-aprovadas-1994344",
    "https://www.publico.pt/2023/02/10/politica/noticia/contingentes-imigrantes-suspensos-decisao-governo-moedas-fez-parte-2038395",
    "https://www.publico.pt/2022/06/25/azul/noticia/verdade-incendios-sao-principal-fonte-emissoes-carbono-portugal-2011329",
    "https://www.publico.pt/2021/03/12/politica/noticia/sair-pais-partir-segundafeira-1954172",
    "https://www.publico.pt/2022/05/20/sociedade/noticia/jose-socrates-ausentarse-pais-comunicar-tribunal-2007031",
    "https://www.publico.pt/2021/12/28/politica/noticia/rui-rio-propos-acordo-parlamentar-dois-anos-ps-argumenta-proprio-1990117",
    "https://www.publico.pt/2022/07/18/desporto/noticia/carro-cristiano-ronaldo-recentemente-estacionado-alvalade-2014101",
    "https://www.publico.pt/2022/09/22/mundo/noticia/companhias-aereas-russas-obrigadas-parar-vender-bilhetes-homens-18-65-anos-2021523",
    "https://www.publico.pt/2021/07/03/politica/noticia/governo-nao-diminuiu-preco-portagens-lei-1968888",
    "https://www.publico.pt/2023/02/23/economia/noticia/mercadona-oferecer-vouchers-valor-500-euros-2040046",
    "https://www.publico.pt/2021/09/05/sociedade/noticia/formacao-medicos-familia-reino-unido-menos-exigente-noutras-especialidades-1976381",
    "https://www.publico.pt/2020/11/28/politica/noticia/verdade-rio-mudou-posicao-portagens-1940961",
    "https://www.publico.pt/2020/11/21/sociedade/noticia/governo-vacina-nao-1940055",
    "https://www.publico.pt/2019/09/26/politica/noticia/verdade-casa-douro-vai-voltar-sistema-igual-vigorava-tempo-estado-novo-1887974",
    "https://www.publico.pt/2022/11/28/tecnologia/noticia/samsonite-vender-malas-dois-euros-portugal-sair-mercado-russo-2029035",
    "https://www.publico.pt/2022/10/07/desporto/noticia/cartaz-comportamentos-evitar-mundial-2022-qatar-oficial-2023132",
    "https://www.publico.pt/2023/02/03/politica/noticia/so-tres-partidos-ganharam-europeias-governo-2037365",
    "https://www.publico.pt/2021/08/20/mundo/noticia/imagem-aviao-forca-aerea-norteamericana-transportar-centenas-pessoas-afeganistao-verdadeira-1974762",
    "https://www.publico.pt/2020/11/14/politica/noticia/prova-factos-acordo-psd-chega-so-regional-1939196",
    "https://www.publico.pt/2022/09/02/sociedade/noticia/taxa-cobertura-creches-portugal-positiva-comparada-resto-europa-2019207",
    "https://www.publico.pt/2021/04/17/politica/noticia/enriquecimento-ilicito-proposto-pcp-contou-votos-ps-psd-cds-1958835",
    "https://www.publico.pt/2022/01/05/politica/noticia/costa-preferiu-negociar-geringonca-nao-psd-orcamentos-estado-1990813",
    "https://www.publico.pt/2021/10/26/politica/noticia/orcamento-estado-2022-apenas-artigo-educacao-andre-ventura-1982628",
    "https://www.publico.pt/2021/11/09/politica/noticia/forcas-armadas-detectaram-suspeitas-republica-centroafricana-1984235",
    "https://www.publico.pt/2021/06/19/local/noticia/governo-civil-lisboa-nao-enviava-dados-manifestantes-embaixadas-1967078",
    "https://www.publico.pt/2021/12/31/politica/noticia/catarina-martins-primeira-sugerir-acordo-esquerdas-governacao-1990338",
    "https://www.publico.pt/2021/05/02/sociedade/noticia/vamos-chegar-grau-imunizacao-ingleses-menos-tempo-tal-costa-1960852",
    "https://www.publico.pt/2021/01/02/politica/noticia/chefe-casa-militar-marcelo-deixar-cargo-passou-reserva-1944676",
    "https://www.publico.pt/2023/01/27/desporto/noticia/vitoria-clubes-jogos-liga-segundafeira-2036715",
    "https://www.publico.pt/2021/12/10/politica/noticia/renovacao-bancada-parlamentar-rui-rio-1988099",
    "https://www.publico.pt/2022/06/10/economia/noticia/anos-troika-petroleo-caro-combustiveis-baratos-2009644",
    "https://www.publico.pt/2021/04/10/politica/noticia/prova-factos-declaracoes-suzana-garcia-inseremse-pluralidade-psd-1957947"
]

In [112]:
len(urls_to_remove)
for url in urls_to_remove:
    try:
        urls.remove(url)
    except:
        pass

len(urls)

203

In [113]:
# Get first name in text to use as the source
def extract_first_name(text):
    tokens = nltk.word_tokenize(text)
    for i in range(len(tokens) - 1):
        current_token = tokens[i]
        next_token = tokens[i + 1]
        if current_token[0].isupper() and next_token[0].isupper():
            return f"{current_token} {next_token}"
    return None

# Examples
text1 = "Os vários números usados por António Costa nesta afirmação não batem certo com as estatísticas oficiais nem com os números divulgados pelo próprio Governo."
first_name1 = extract_first_name(text1)
print(first_name1)  # Output: António Costa

text2 = "António Costa usou vários números nesta afirmação que não batem certo com as estatísticas oficiais de Oscar Alho nem com os números divulgados pelo próprio Governo."
first_name2 = extract_first_name(text2)
print(first_name2)  # Output: António Costa

António Costa
António Costa


In [114]:
urls_test = [
    "https://www.publico.pt/2021/01/16/politica/noticia/rui-rio-defendeu-abertura-restaurantes-13h-finsdesemana-1946516",
    #"https://www.publico.pt/2023/04/14/azul/noticia/portugal-maior-reserva-litio-europa-2046111",
    #"https://www.publico.pt/2019/09/13/politica/noticia/olival-intensivo-ocupa-apenas-15-conjunto-area-alentejo-1886552",
    #"https://www.publico.pt/2020/12/25/sociedade/noticia/verdade-ja-nao-aeroporto-inspectores-sef-visados-relatorio-igai-1944175",
    #"https://www.publico.pt/2023/01/10/politica/noticia/imagem-lula-silva-algemado-partilhada-andre-ventura-verdadeira-2034525",
    #"https://www.publico.pt/2023/03/11/politica/noticia/ps-eleitoralmente-forte-dentro-geringonca-psd-face-il-chega-2041931"
    #"https://www.publico.pt/2021/09/03/politica/noticia/so-oito-paises-touradas-1976186",
    # "https://www.publico.pt/2022/07/08/politica/noticia/psd-votou-favor-mocoes-censura-cds-andre-ventura-2012933?ref=prova-dos-factos&cx=stack",
    # "https://www.publico.pt/2023/06/15/economia/noticia/cgd-pagaram-milhao-euros-exadministrador-750-mil-euros-pedro-nuno-santos-2053457",
    # "https://www.publico.pt/2023/06/22/ciencia/noticia/simpsons-previram-desaparecimento-submarino-titan-2054255",
    # "https://www.publico.pt/2023/05/16/sociedade/noticia/vaticano-lancou-selo-comemorativo-jmj-padrao-descobrimentos-2049786",
    #"https://www.publico.pt/2023/06/06/p3/noticia/sexo-passou-modalidade-desportiva-suecia-2052434",
]

In [115]:
            # Get as_frases label, which seems to be a special case
            # if as_frases_label == "NONE":
            #     as_frases_label_element = soup.find("div", class_ = "image-wrapper")
            #     as_frases_label = as_frases_label_element.get_text()
            #     if as_frases_label != "Assine já":
            #         label = as_frases_label
            #         label_check = False
            # else:
            #     label = as_frases_label

In [116]:
failed_label_urls = set()
image_urls = set()
post_urls = set()
texts=[]
sources=[]
labels = []
df_urls = []
# Initialize an empty dataframe
df = pd.DataFrame(columns=['Text', 'Source', 'Label', 'URL'])

# We should try finding the text that is being fact-checked, along with its source
for url in urls:
    label_check = True
    label_check_tries = 0
    as_frases_label = "NONE"
    while(label_check):
        if label_check_tries == 20:
            print("Max label tries achieved!!!")
            failed_label_urls.add(url)
            break
        print("-------------")
        print(url)
        print("-------------")
        text = "NONE"
        source = "NONE"
        label = "NONE"
        a_frase = False
        a_frase_paragraph = False
        a_publicacao = False
        a_imagem = False
        o_contexto = False
        label_check_tries_incremented = False
        as_frases_contexto_found = False
        statements_list = []
        sources_list = []
        try:
            driver.get(url)
            soup = bs4(driver.page_source, 'html.parser')
            #print("OMG",soup.find("div", class_="story__blurb lead"))            
            h2_elements = soup.find_all("h2")
            for element in h2_elements:
                if "frase" in element.get_text().lower():  # A frase
                    a_frase = True
                elif "publicação" in element.get_text().lower(): # A publicação
                    a_publicacao = True
                elif "imagem" in element.get_text().lower(): # A imagem
                    a_imagem = True
                elif "contexto" in element.get_text().lower():  # O Context
                    o_contexto = True
                    contexto_element = element
            if not(a_frase):
                # Sometimes "A frase" is tagged as a p element instead of an h2 element
                paragraphs = soup.find_all("p")
                for p in paragraphs:
                    #print(p)
                    if p.get_text() == "A frase":
                        a_frase_paragraph = True
                        a_frase_paragraph_element = p
            if a_frase:
                print("a_frase")
                blockquote_element = soup.find("blockquote") # typically the text has quotation marks
                if blockquote_element:
                    text = soup.find("blockquote").get_text().strip()
                    lines = text.splitlines()
                    text = lines[0] # first line contains the original sentence
                    source = lines[-1]  # last line usually contains the source that said it
                    if source == text:
                        if "A frase" in source:
                            # The source extraction was unsucessful, so we resort to the figcaption
                            text = lines[0].split("A frase")[1]
                            #print(soup.find("figcaption", class_ = "caption caption--image"))
                            source = soup.find("figcaption", class_ = "caption caption--image").get_text()
                            source = source.strip()
                            lines = source.splitlines()

                            # Join the lines back into a string
                            source = '\n'.join(lines).lstrip()
                            source = re.sub(r'\n+', '\n', source) # Replace consecutive paragraphs with a single one
                            source = re.sub(r'\t', '', source) # Replace consecutive paragraphs with a single one

                            lines = source.splitlines()
                            source = extract_first_name(lines[0])
                        else:
                            text = lines[0]
                            source = text.split(".")[-1]
                            text = text.replace(source, "")
                    if "saiba mais" in source.lower():
                        blockquote_element = soup.find("blockquote")
                        source = blockquote_element.find_next_sibling().get_text() #other times it's the next element that has the source
                else:
                    body_element = soup.find("div", class_ = "story__body") # when it doesn't have quotation marks, we get the first paragraph
                    paragraphs = body_element.find_all("p")
                    text = paragraphs[0].get_text().strip()
                    source_paragraph = paragraphs[-1].get_text().strip()
                    source = extract_first_name(source_paragraph) # get first name in text to use as the source
                    if not(source): # reset source if it is None
                        source = "NONE"
            elif a_frase_paragraph:
                paragraphs = soup.find_all("p")
                # for p in paragraphs:
                #     print(p)
                #print("\n",a_frase_paragraph_element)
                text_element = a_frase_paragraph_element.find_next_sibling().find_next_sibling().find_next_sibling().find_next_sibling()
                #print(text_element)
                text = text_element.get_text().strip()
                source = text.split("”")[-1]
                text = text.replace(source, "")
                source = re.sub(r'(?<=[a-z])([A-Z])', r' \1', source)
            # elif a_publicacao:
            #     print("a_publicacao")
            #     text = "PUBLICAÇÃO"
            #     source = "add source of a_publicacao"
            # elif a_imagem:
            #     print("a_imagem")
            #     text = "IMAGEM"
            #     image_urls.add(url)
            elif o_contexto:
                print("o_contexto")
                text = ""
                contexto_text = ""

                # Get the first two paragraphs
                body_element = soup.find("div", class_ = "story__body")
                paragraphs = body_element.find_all("p")
                # for p in paragraphs:
                #     print(p)
                p0_first_char = ""
                if paragraphs[0].get_text():
                    p0_first_char = paragraphs[0].get_text()[0]
                #print("p0_first_char",p0_first_char)
                p4_first_char = ""
                if paragraphs[4].get_text():
                    p4_first_char = paragraphs[4].get_text()[0]
                #print("p4_first_char",p4_first_char)
                
                if paragraphs[0].get_text() and "“" in p0_first_char:
                    contexto_text = paragraphs[0].get_text() #for some reason, paragraphs[0] + "\n" + paragraphs[1].get_text() is bad
                    contexto_text += '\n' + paragraphs[3].get_text()
                elif "“" in p4_first_char:
                    contexto_text = paragraphs[4].get_text()
                elif "title-subscription-disclaimer" in str(paragraphs[1]):
                    contexto_text = paragraphs[3].get_text()
                    contexto_text += '\n' + paragraphs[4].get_text()
                else:
                    contexto_text = paragraphs[1].get_text()
                    contexto_text += '\n' + paragraphs[2].get_text()

                #print(contexto_text)
                # Extract text delimited by quotation marks
                pattern = r'[“”"](.*?)[“”"]'
                matches = re.findall(pattern, contexto_text)
                for match in matches:
                    print("match",match)
                    if len(match) > 23:
                        #print("match",match)
                        text += match + "\n"

                text = text.rstrip() # remove last paragraph
                #print(contexto_text)
                if source == "NONE":
                    if "facebook" in contexto_text.lower():
                        source = "facebook"
                    elif "twitter" in contexto_text.lower():
                        source = "twitter"
                    elif "redes sociais" in contexto_text.lower():
                        source = "redes sociais"
                    else:
                        source = "n/a"

                if text == "":
                    print("CONTEXTO Failed, the title and text will be used as the statement")
                    text = soup.find("h1", class_ = "headline story__headline").get_text().strip()
                    text = text.replace("?", ".")
                    text += "\n" + contexto_text
            else:
                print("else")
                # Use the title as the statement and replace '?' with '.', unless "As frases" is found in the text
                body_element = soup.find("div", class_ = "story__body")
                paragraphs = body_element.find_all("p")
                as_frases_found = False
                for p in paragraphs:
                    #print(p)
                    if p.get_text() == "As frases":
                        print("as frases found")
                        as_frases_element = p
                        as_frases_found = True
                if as_frases_found:
                    statements_list = []
                    sources_list = []
                    as_frases_contexto_found = False
                    next_element = as_frases_element
                    print("starting element",next_element)
                    while not(as_frases_contexto_found):
                        if next_element.get_text():
                            statements_list.append(next_element.get_text().strip())
                            print("as frases statement",next_element.get_text().strip())
                        next_element = next_element.find_next_sibling()
                        if next_element.get_text():
                            print("as frases source",next_element.get_text().strip())
                            sources_list.append(next_element.get_text().strip())
                        next_element = next_element.find_next_sibling()
                        print("next element after cycle text",next_element.get_text().strip())
                        if next_element.get_text().strip() == "O contexto":
                            print("as frases contexto found")
                            as_frases_contexto_found = True
                            # the first position is not relevant
                            statements_list.pop(0)
                            sources_list.pop(0)
                else:
                    text = soup.find("h1").get_text().strip()
                    text = text.replace("?", ".")
                    source = "n/a"

            if a_publicacao:
                post_urls.add(url)

            if a_imagem:
                # The image could be relevant to train fake image or deep fake detection models
                image_urls.add(url)
                if source == "NONE" or source == "n/a":
                    paragraphs = soup.find_all("p")
                    source = extract_first_name(paragraphs[2].get_text().strip())

            label = soup.find("div", class_ = "image-wrapper")
            #print("label",label)
            image_label_element = soup.find("figure", class_ = "story__callout story__callout--image story__callout--image_svg media media--image story__callout--inline")
            #print("image_label_element",image_label_element)
            if label:
                print("label found")
                label = label.get_text()
                if label != "Assine já":
                    label_check = False
                else:
                    label_check_tries+=1
                    print("label tries",label_check_tries)
            elif image_label_element:
                print("image_label_element found")
                flex_media_element = soup.find('div', class_ = "flex-media svg")
                #print(flex_media_element)
                temp_list = []
                for element in flex_media_element:
                    temp_list.append(str(element))
                for string_element in temp_list:
                    if "data-media-viewer=" in string_element:
                        #print("string_elem",string_element)
                        pattern = r'src="([^"]*)"'
                        match = re.search(pattern, string_element)
                        if match:
                            link = match.group(1)
                            #print(link)
                            link_label = link.split("/")[-1].split(".")[0]
                            if link_label == "nao" or link_label == "talvez":
                                label = "Falso"
                            else:
                                label == "Verdadeiro"
                            label_check = False
                        else:
                            print("No link found")
                            label_check_tries+=1
                    else:
                        label_check_tries+=1
            else:
                label_check_tries+=1
                label_check_tries_incremented = True
                print("label tries",label_check_tries)

        except:
            pass
        
        # print("len statements list",len(statements_list))
        # print("len sources list",len(sources_list))
        # for element in statements_list:
        #     print(element)
        # for element in sources_list:
        #     print(element)

        if (label and label != "NONE") or label_check_tries == 20:
            if label_check_tries == 20:
                print("failed url added")
                failed_label_urls.add(url)
            elif statements_list or sources_list:
                while statements_list or sources_list:
                    as_frases_text = statements_list.pop(0)
                    as_frases_source = sources_list.pop(0)
                    texts.append(as_frases_text)
                    sources.append(as_frases_source)
                    labels.append(label)
                    df_urls.append(url)
                    print(as_frases_text)
                    print(as_frases_source)
                    print(label)
            elif text != "PÚBLICO" and label != "Assine já":
                print("appended")
                texts.append(text)
                sources.append(source)
                labels.append(label)
                df_urls.append(url)

        print(text)
        print(source)
        print(label)

        time.sleep(3)
        
# Create a new dataframe from the lists
new_data = pd.DataFrame({'Text': texts, 'Source': sources, 'Label': labels, 'URL': df_urls})

# Concatenate the new dataframe with the existing dataframe
df = pd.concat([df, new_data], ignore_index=True)

-------------
https://www.publico.pt/2023/01/10/politica/noticia/imagem-lula-silva-algemado-partilhada-andre-ventura-verdadeira-2034525
-------------


o_contexto
match Compreendo a dor e a revolta de milhões ao verem, de repente, que são governados por um bandido ex-presidiário! Sinto o mesmo! Mas a democracia tem regras próprias e é na arena democrática que o combate tem de ser travado, não com violência!
label found
appended
Compreendo a dor e a revolta de milhões ao verem, de repente, que são governados por um bandido ex-presidiário! Sinto o mesmo! Mas a democracia tem regras próprias e é na arena democrática que o combate tem de ser travado, não com violência!
André Ventura
Falso
-------------
https://www.publico.pt/2022/12/09/sociedade/noticia/sns-actualmente-medicos-especialistas-final-2015-2030840
-------------
a_frase
label tries 1
“O SNS tem hoje cerca de 4300 médicos especialistas mais do que tinha no final de 2015”.
Manuel Pizarro, ministro da Saúde
None
-------------
https://www.publico.pt/2022/12/09/sociedade/noticia/sns-actualmente-medicos-especialistas-final-2015-2030840
-------------
a_frase
label tries 2
“O SNS tem h

In [117]:
len(urls)

203

In [118]:
df.head()
df.to_csv("Publico_fact_check_dataset_p1.csv", index=False)

In [119]:
loaded_df = pd.read_csv("Publico_fact_check_dataset_p1.csv")
loaded_df.head()

,Text,Source,Label,URL
0,Compreendo a dor e a revolta de milhões ao ver...,André Ventura,Falso,https://www.publico.pt/2023/01/10/politica/not...
1,“O SNS tem hoje cerca de 4300 médicos especial...,"Manuel Pizarro, ministro da Saúde",Verdadeiro,https://www.publico.pt/2022/12/09/sociedade/no...
2,"""Não tinha de me pronunciar nem ter conhecimen...","Marcelo Rebelo de Sousa, Presidente da República",Verdadeiro,https://www.publico.pt/2023/03/03/politica/not...
3,Vídeo de pára-quedistas russos a aterrarem em ...,twitter,Falso,https://www.publico.pt/2022/02/24/mundo/notici...
4,“Eu creio que com excepção do Aeroporto da Mad...,António Costa,Falso,https://www.publico.pt/2021/09/24/politica/not...


In [120]:
for source in set(loaded_df['Source']):
    print(source)

nan
Cristina Rodrigues
Nuno Melo, presidente do CDS
Pedro Nuno Santos, ministro das Infra-Estruturas e Habitação
“Temos conhecimento de que Espanha está a fechar a torneira aos rios portugueses”, Mariana Mortágua, deputada do Bloco de Esquerda
Comunicado do Conselho Nacional das Ordens Profissionais (CNOP), 16 de Fevereiro de 2023
André Ventura, presidente do Chega
José Gouveia, presidente da Associação Nacional de Discotecas, no dia 30 de Setembro
Paulo Rangel, em entrevista à RR dia 15
António Costa, no domingo, na SIC, durante o frente-a-frente com o líder do CDS, Francisco Rodrigues dos Santos
João Pedro Matos Fernandes, ministro do Ambiente e da Acção Climática em entrevista ao Expresso
Carla Castro, candidata à presidência da comissão executiva da Iniciativa Liberal
Elon Musk, líder da Tesla e da SpaceX
Site oficial da ponte 516 Arouca
António Costa, Twitter, 8 de Maio
Carlos Guimarães Pinto, deputado da Iniciativa Liberal
Mário Nogueira, secretário-geral da Fenprof
Hugo Carneiro

In [121]:
# urls to check
urls_to_check = [
    "https://www.publico.pt/2023/04/07/politica/noticia/recibos-verdes-estado-aumentaram-14-2045319",
    "https://www.publico.pt/2023/04/04/impar/noticia/politica-francesa-marlene-schiappa-posou-nua-lingerie-playboy-2044934",
    "https://www.publico.pt/2021/06/11/economia/noticia/ana-paula-vitorino-ira-salario-maior-pedro-adao-silva-rui-rio-1966168",
    "https://www.publico.pt/2021/09/13/desporto/noticia/treinador-dinamo-kiev-mircea-lucescu-treinador-titulos-ganhos-actividade-1977311",
    "https://www.publico.pt/2022/07/13/azul/noticia/unico-responsavel-incendios-accao-humana-antonio-costa-2013363",
    "https://www.publico.pt/2023/06/01/azul/noticia/grupo-internacional-activistas-esvazia-pneus-carros-suv-chegou-portugal-2051821",
    "https://www.publico.pt/2022/02/04/politica/noticia/candidaturas-vicepresidentes-pe-aprovadas-1994344",
    "https://www.publico.pt/2023/02/10/politica/noticia/contingentes-imigrantes-suspensos-decisao-governo-moedas-fez-parte-2038395",
    "https://www.publico.pt/2022/06/25/azul/noticia/verdade-incendios-sao-principal-fonte-emissoes-carbono-portugal-2011329",
    "https://www.publico.pt/2021/03/12/politica/noticia/sair-pais-partir-segundafeira-1954172",
    "https://www.publico.pt/2022/05/20/sociedade/noticia/jose-socrates-ausentarse-pais-comunicar-tribunal-2007031",
    "https://www.publico.pt/2021/12/28/politica/noticia/rui-rio-propos-acordo-parlamentar-dois-anos-ps-argumenta-proprio-1990117",
    "https://www.publico.pt/2022/07/18/desporto/noticia/carro-cristiano-ronaldo-recentemente-estacionado-alvalade-2014101",
    "https://www.publico.pt/2022/09/22/mundo/noticia/companhias-aereas-russas-obrigadas-parar-vender-bilhetes-homens-18-65-anos-2021523",
    "https://www.publico.pt/2021/07/03/politica/noticia/governo-nao-diminuiu-preco-portagens-lei-1968888",
    "https://www.publico.pt/2023/02/23/economia/noticia/mercadona-oferecer-vouchers-valor-500-euros-2040046",
    "https://www.publico.pt/2021/09/05/sociedade/noticia/formacao-medicos-familia-reino-unido-menos-exigente-noutras-especialidades-1976381",
    "https://www.publico.pt/2020/11/28/politica/noticia/verdade-rio-mudou-posicao-portagens-1940961",
    "https://www.publico.pt/2020/11/21/sociedade/noticia/governo-vacina-nao-1940055",
    "https://www.publico.pt/2019/09/26/politica/noticia/verdade-casa-douro-vai-voltar-sistema-igual-vigorava-tempo-estado-novo-1887974",
    "https://www.publico.pt/2022/11/28/tecnologia/noticia/samsonite-vender-malas-dois-euros-portugal-sair-mercado-russo-2029035",
    "https://www.publico.pt/2022/10/07/desporto/noticia/cartaz-comportamentos-evitar-mundial-2022-qatar-oficial-2023132",
    "https://www.publico.pt/2023/02/03/politica/noticia/so-tres-partidos-ganharam-europeias-governo-2037365",
    "https://www.publico.pt/2021/08/20/mundo/noticia/imagem-aviao-forca-aerea-norteamericana-transportar-centenas-pessoas-afeganistao-verdadeira-1974762",
    "https://www.publico.pt/2020/11/14/politica/noticia/prova-factos-acordo-psd-chega-so-regional-1939196",
    "https://www.publico.pt/2022/09/02/sociedade/noticia/taxa-cobertura-creches-portugal-positiva-comparada-resto-europa-2019207",
    "https://www.publico.pt/2021/04/17/politica/noticia/enriquecimento-ilicito-proposto-pcp-contou-votos-ps-psd-cds-1958835",
    "https://www.publico.pt/2022/01/05/politica/noticia/costa-preferiu-negociar-geringonca-nao-psd-orcamentos-estado-1990813",
    "https://www.publico.pt/2021/10/26/politica/noticia/orcamento-estado-2022-apenas-artigo-educacao-andre-ventura-1982628",
    "https://www.publico.pt/2021/11/09/politica/noticia/forcas-armadas-detectaram-suspeitas-republica-centroafricana-1984235",
    "https://www.publico.pt/2021/06/19/local/noticia/governo-civil-lisboa-nao-enviava-dados-manifestantes-embaixadas-1967078",
    "https://www.publico.pt/2021/12/31/politica/noticia/catarina-martins-primeira-sugerir-acordo-esquerdas-governacao-1990338",
    "https://www.publico.pt/2021/05/02/sociedade/noticia/vamos-chegar-grau-imunizacao-ingleses-menos-tempo-tal-costa-1960852",
    "https://www.publico.pt/2021/01/02/politica/noticia/chefe-casa-militar-marcelo-deixar-cargo-passou-reserva-1944676",
    "https://www.publico.pt/2023/01/27/desporto/noticia/vitoria-clubes-jogos-liga-segundafeira-2036715",
    "https://www.publico.pt/2021/12/10/politica/noticia/renovacao-bancada-parlamentar-rui-rio-1988099",
    "https://www.publico.pt/2022/06/10/economia/noticia/anos-troika-petroleo-caro-combustiveis-baratos-2009644",
    "https://www.publico.pt/2021/04/10/politica/noticia/prova-factos-declaracoes-suzana-garcia-inseremse-pluralidade-psd-1957947"
]

In [122]:
failed_label_urls = set()
image_urls = set()
post_urls = set()
texts=[]
sources=[]
labels = []
df_urls = []
# Initialize an empty dataframe
df = pd.DataFrame(columns=['Text', 'Source', 'Label', 'URL'])

# We should try finding the text that is being fact-checked, along with its source
for url in urls_to_check:
    label_check = True
    label_check_tries = 0
    as_frases_label = "NONE"
    while(label_check):
        if label_check_tries == 20:
            print("Max label tries achieved!!!")
            failed_label_urls.add(url)
            break
        print("-------------")
        print(url)
        print("-------------")
        text = "NONE"
        source = "NONE"
        label = "NONE"
        a_frase = False
        a_frase_paragraph = False
        a_publicacao = False
        a_imagem = False
        o_contexto = False
        label_check_tries_incremented = False
        as_frases_contexto_found = False
        statements_list = []
        sources_list = []
        try:
            driver.get(url)
            soup = bs4(driver.page_source, 'html.parser')
            #print("OMG",soup.find("div", class_="story__blurb lead"))            
            h2_elements = soup.find_all("h2")
            for element in h2_elements:
                print(element)
                if "frase" or "frase:" in element.get_text().lower():  # A frase
                    a_frase = True
                elif "publicação" in element.get_text().lower(): # A publicação
                    a_publicacao = True
                elif "imagem" in element.get_text().lower(): # A imagem
                    a_imagem = True
                elif "contexto" in element.get_text().lower():  # O Context
                    o_contexto = True
                    contexto_element = element
            if not(a_frase):
                # Sometimes "A frase" is tagged as a p element instead of an h2 element
                paragraphs = soup.find_all("p")
                for p in paragraphs:
                    #print(p)
                    if p.get_text() == "A frase":
                        a_frase_paragraph = True
                        a_frase_paragraph_element = p
            if a_frase:
                print("a_frase")
                blockquote_element = soup.find("blockquote") # typically the text has quotation marks
                blockquote_from_twitter = soup.find('blockquote', class_='twitter-tweet')
                if blockquote_element and not(blockquote_from_twitter):
                    #print(blockquote_element)
                    text = soup.find("blockquote").get_text().strip()
                    lines = text.splitlines()
                    #print(lines)
                    text = lines[0] # first line contains the original sentence
                    source = lines[-1]  # last line usually contains the source that said it
                    print("HI blockquote")
                    print(text)
                    print(source)
                    if source == text:
                        if "A frase" in source:
                            print("A frase in source")
                            # The source extraction was unsucessful, so we resort to the figcaption
                            text = lines[0].split("A frase")[1]
                            #print(soup.find("figcaption", class_ = "caption caption--image"))
                            source = soup.find("figcaption", class_ = "caption caption--image").get_text()
                            source = source.strip()
                            lines = source.splitlines()

                            # Join the lines back into a string
                            source = '\n'.join(lines).lstrip()
                            source = re.sub(r'\n+', '\n', source) # Replace consecutive paragraphs with a single one
                            source = re.sub(r'\t', '', source) # Replace consecutive paragraphs with a single one

                            lines = source.splitlines()
                            source = extract_first_name(lines[0])
                        else:
                            text = lines[0]
                            source = text.split(".")[-1]
                            text = text.replace(source, "")
                            if source == "”":
                                text = soup.find("blockquote").get_text().strip()
                                source = soup.find("blockquote").find_next_sibling().get_text().strip()
                            elif "”" in source:
                                text = lines[0].split("”")[0]
                                source = lines[0].split("”")[-1]
                            elif not(source):
                                paragraphs = soup.find_all("p")
                                source_efn = extract_first_name(paragraphs[2].get_text())
                                #print("source_efn",source_efn)
                                if "facebook" in paragraphs[2].get_text().strip().lower():
                                    source = "facebook"
                                elif "twitter" in paragraphs[2].get_text().strip().lower():
                                    source = "twitter"
                                elif "redes sociais" in paragraphs[2].get_text().strip().lower():
                                    source = "redes sociais"
                                elif source_efn:
                                    source = source_efn
                    if "saiba mais" in source.lower():
                        blockquote_element = soup.find("blockquote")
                        source = blockquote_element.find_next_sibling().get_text() #other times it's the next element that has the source
                    if source.strip() == "O contexto" or source.strip() == "O Contexto":
                        print("o contexto blockquote")
                        old_text = text
                        if '"' in old_text:
                            quote_index = old_text.rindex('"')
                            text = old_text[:quote_index + 1]
                            source = old_text[quote_index + 1:]
                        elif "”" in old_text:
                            quote_index = old_text.rindex('”')
                            text = old_text[:quote_index + 1]
                            source = old_text[quote_index + 1:]
                        elif "“" in old_text and "”" not in old_text:
                            text = old_text
                            source = extract_first_name(soup.find("h1", class_ = "headline story__headline").get_text().strip())
                        else:
                            source = text.split(".")[-1]
                            text = text.replace(source, "")
                        print(text)
                        print(source)
                        if text == '"' or source == "O contexto":
                            text = lines[0].split("”")[0]
                            source = lines[0].split("”")[-1]
                        if source == "":
                            paragraphs = soup.find_all("p")
                            # for p in paragraphs:
                            #     print(p)
                            source_efn = extract_first_name(paragraphs[2].get_text())
                            print("source_efn",source_efn)
                            if "facebook" in paragraphs[2].get_text().strip().lower():
                                source = "facebook"
                            elif "twitter" in paragraphs[2].get_text().strip().lower():
                                source = "twitter"
                            elif "redes sociais" in paragraphs[2].get_text().strip().lower():
                                source = "redes sociais"
                            elif source_efn:
                                source = source_efn
                            else:
                                source = soup.find("figcaption", class_ = "caption caption--image").get_text()
                                source = source.strip()
                                lines = source.splitlines()

                                # Join the lines back into a string
                                source = '\n'.join(lines).lstrip()
                                source = re.sub(r'\n+', '\n', source) # Replace consecutive paragraphs with a single one
                                source = re.sub(r'\t', '', source) # Replace consecutive paragraphs with a single one

                                lines = source.splitlines()
                                source = extract_first_name(lines[0])
                        if source == ".":
                            paragraphs = soup.find_all("p")
                            source_efn = extract_first_name(paragraphs[2].get_text())
                            #print("source_efn",source_efn)
                            if "facebook" in paragraphs[2].get_text().strip().lower():
                                source = "facebook"
                            elif "twitter" in paragraphs[2].get_text().strip().lower():
                                source = "twitter"
                            elif "redes sociais" in paragraphs[2].get_text().strip().lower():
                                source = "redes sociais"
                            elif source_efn:
                                source = source_efn
                    elif source.strip() == "O contexto:":
                        headline_text = soup.find("h1", class_ = "headline story__headline").get_text().strip()
                        source = headline_text.split("disse")[-1]
                        source = source[:-1].strip()

                else:
                    body_element = soup.find("div", class_ = "story__body") # when it doesn't have quotation marks, we get the first paragraph
                    paragraphs = body_element.find_all("p")
                    for p in paragraphs:
                        print(p)
                    text = paragraphs[0].get_text().strip()
                    source_paragraph = paragraphs[-1].get_text().strip()
                    source = extract_first_name(source_paragraph) # get first name in text to use as the source
                    if not(source): # reset source if it is None
                        source = "NONE"
                    if text == "A frase":
                        text = paragraphs[3].get_text().strip()
                        source = paragraphs[4].get_text().strip()
                    if source == "NONE":
                        if "facebook" in text.lower():
                            source = "facebook"
                        elif "twitter" in text.lower():
                            source = "twitter"
                        elif "redes sociais" in text.lower():
                            source = "redes sociais"
                        else:
                            headline_text = soup.find("h1", class_ = "headline story__headline").get_text().strip()
                            text = headline_text
                            text = text.replace("?", ".")
                            text += "\n" + paragraphs[3].get_text().strip() + "\n" + paragraphs[4].get_text().strip()
                            if paragraphs[4].find_next_sibling().get_text() != "Os factos":
                               text += "\n" + paragraphs[5].get_text().strip()
                            
                            if "facebook" in text.lower():
                                source = "facebook"
                            elif "twitter" in text.lower():
                                source = "twitter"
                            elif "redes sociais" in text.lower():
                                source = "redes sociais"
                            else:
                                paragraphs = soup.find_all("p")
                                # for p in paragraphs:
                                #     print(p)
                                if "Fonte:" in paragraphs[4].get_text().strip():
                                    source = paragraphs[4].get_text().split("Fonte:")[-1].strip()
                                # elif len(paragraphs[3].get_text()) < 34:
                                #     text = paragraphs[0].get_text().strip()
                                #     source = paragraphs[3].get_text().strip()
                                else:
                                    paragraphs = body_element.find_all("p")
                                    # for p in paragraphs:
                                    #     print(p)
                                    if "”" in paragraphs[0].get_text().strip():
                                        text = paragraphs[0].get_text().strip()
                                        if paragraphs[1].get_text().strip() == "Leia os artigos que quiser, até ao fim.":
                                            if text[-1] == "”":
                                                source = text.split("”")[-1].strip()
                                            elif len(text.split("”")[-1]) > 2:
                                                source = text.split("”")[-1].strip()
                                            else:
                                                source = paragraphs[3].get_text().strip()
                                        else:
                                            source = "n/a"
                                    elif '"' in paragraphs[0].get_text().strip():
                                        text = paragraphs[0].get_text().split('"')[1]
                                        source = paragraphs[0].get_text().split('"')[-1]
                                    else:
                                        source = "n/a"
                    if source == "O contexto":
                        paragraphs = soup.find_all("p")
                        # for p in paragraphs:
                        #     print(p)
                        source = source_efn = extract_first_name(paragraphs[2].get_text())
            elif a_frase_paragraph:
                paragraphs = soup.find_all("p")
                # for p in paragraphs:
                #     print(p)
                #print("\n",a_frase_paragraph_element)
                text_element = a_frase_paragraph_element.find_next_sibling().find_next_sibling().find_next_sibling().find_next_sibling()
                #print(text_element)
                text = text_element.get_text().strip()
                source = text.split("”")[-1]
                text = text.replace(source, "")
                source = re.sub(r'(?<=[a-z])([A-Z])', r' \1', source)
            # elif a_publicacao:
            #     print("a_publicacao")
            #     text = "PUBLICAÇÃO"
            #     source = "add source of a_publicacao"
            # elif a_imagem:
            #     print("a_imagem")
            #     text = "IMAGEM"
            #     image_urls.add(url)
            elif o_contexto:
                print("o_contexto")
                text = ""
                contexto_text = ""

                # Get the first two paragraphs
                body_element = soup.find("div", class_ = "story__body")
                paragraphs = body_element.find_all("p")
                # for p in paragraphs:
                #     print(p)
                p0_first_char = ""
                if paragraphs[0].get_text():
                    p0_first_char = paragraphs[0].get_text()[0]
                #print("p0_first_char",p0_first_char)
                p4_first_char = ""
                if paragraphs[4].get_text():
                    p4_first_char = paragraphs[4].get_text()[0]
                #print("p4_first_char",p4_first_char)
                
                if paragraphs[0].get_text() and "“" in p0_first_char:
                    contexto_text = paragraphs[0].get_text() #for some reason, paragraphs[0] + "\n" + paragraphs[1].get_text() is bad
                    contexto_text += '\n' + paragraphs[3].get_text()
                elif "“" in p4_first_char:
                    contexto_text = paragraphs[4].get_text()
                elif "title-subscription-disclaimer" in str(paragraphs[1]):
                    contexto_text = paragraphs[3].get_text()
                    contexto_text += '\n' + paragraphs[4].get_text()
                else:
                    contexto_text = paragraphs[1].get_text()
                    contexto_text += '\n' + paragraphs[2].get_text()

                #print(contexto_text)
                # Extract text delimited by quotation marks
                pattern = r'[“”"](.*?)[“”"]'
                matches = re.findall(pattern, contexto_text)
                for match in matches:
                    print("match",match)
                    if len(match) > 23:
                        #print("match",match)
                        text += match + "\n"

                text = text.rstrip() # remove last paragraph

                if text[0].islower():
                    text = "" # reset text to use the whole paragraph

                #print(contexto_text)
                if source == "NONE":
                    if "facebook" in contexto_text.lower():
                        source = "facebook"
                    elif "twitter" in contexto_text.lower():
                        source = "twitter"
                    elif "redes sociais" in contexto_text.lower():
                        source = "redes sociais"
                    else:
                        source = "n/a"

                if text == "":
                    print("CONTEXTO Failed, the title and text will be used as the statement")
                    text = soup.find("h1", class_ = "headline story__headline").get_text().strip()
                    text = text.replace("?", ".")
                    text += "\n" + contexto_text
            else:
                print("else")
                # Use the title as the statement and replace '?' with '.', unless "As frases" is found in the text
                body_element = soup.find("div", class_ = "story__body")
                paragraphs = body_element.find_all("p")
                as_frases_found = False
                for p in paragraphs:
                    #print(p)
                    if p.get_text() == "As frases":
                        print("as frases found")
                        as_frases_element = p
                        as_frases_found = True
                if as_frases_found:
                    statements_list = []
                    sources_list = []
                    as_frases_contexto_found = False
                    next_element = as_frases_element
                    print("starting element",next_element)
                    while not(as_frases_contexto_found):
                        if next_element.get_text():
                            statements_list.append(next_element.get_text().strip())
                            print("as frases statement",next_element.get_text().strip())
                        next_element = next_element.find_next_sibling()
                        if next_element.get_text():
                            print("as frases source",next_element.get_text().strip())
                            sources_list.append(next_element.get_text().strip())
                        next_element = next_element.find_next_sibling()
                        print("next element after cycle text",next_element.get_text().strip())
                        if next_element.get_text().strip() == "O contexto":
                            print("as frases contexto found")
                            as_frases_contexto_found = True
                            # the first position is not relevant
                            statements_list.pop(0)
                            sources_list.pop(0)
                else:
                    text = soup.find("h1").get_text().strip()
                    text = text.replace("?", ".")
                    source = "n/a"

            if a_publicacao:
                post_urls.add(url)

            if a_imagem:
                # The image could be relevant to train fake image or deep fake detection models
                image_urls.add(url)
                if source == "NONE" or source == "n/a":
                    paragraphs = soup.find_all("p")
                    source = extract_first_name(paragraphs[2].get_text().strip())

            label = soup.find("div", class_ = "image-wrapper")
            #print("label",label)
            image_label_element = soup.find("figure", class_ = "story__callout story__callout--image story__callout--image_svg media media--image story__callout--inline")
            #print("image_label_element",image_label_element)
            if label:
                print("label found")
                label = label.get_text()
                if label != "Assine já":
                    label_check = False
                else:
                    label_check_tries+=1
                    print("label tries",label_check_tries)
            elif image_label_element:
                print("image_label_element found")
                flex_media_element = soup.find('div', class_ = "flex-media svg")
                #print(flex_media_element)
                temp_list = []
                for element in flex_media_element:
                    temp_list.append(str(element))
                for string_element in temp_list:
                    if "data-media-viewer=" in string_element:
                        #print("string_elem",string_element)
                        pattern = r'src="([^"]*)"'
                        match = re.search(pattern, string_element)
                        if match:
                            link = match.group(1)
                            #print(link)
                            link_label = link.split("/")[-1].split(".")[0]
                            if link_label == "nao" or link_label == "talvez":
                                label = "Falso"
                            else:
                                label == "Verdadeiro"
                            label_check = False
                        else:
                            print("No link found")
                            label_check_tries+=1
                    else:
                        label_check_tries+=1
            else:
                label_check_tries+=1
                label_check_tries_incremented = True
                print("label tries",label_check_tries)

        except:
            pass
        
        # print("len statements list",len(statements_list))
        # print("len sources list",len(sources_list))
        # for element in statements_list:
        #     print(element)
        # for element in sources_list:
        #     print(element)
        # try:
        #     label_try= soup.find("div", class_ = "image-wrapper")
        #     print("label",label_try)
        #     if label_try:
        #         print("label found")
        #         label = label_try.get_text()
        #         if label != "Assine já":
        #             label_check = False
        # except:
        #     pass

        if (label and label != "NONE") or label_check_tries == 20:
            if label_check_tries == 20:
                print("failed url added")
                failed_label_urls.add(url)
            elif statements_list or sources_list:
                while statements_list or sources_list:
                    as_frases_text = statements_list.pop(0)
                    as_frases_source = sources_list.pop(0)
                    texts.append(as_frases_text)
                    sources.append(as_frases_source)
                    labels.append(label)
                    df_urls.append(url)
                    print(as_frases_text)
                    print(as_frases_source)
                    print(label)
            elif text != "PÚBLICO" and label != "Assine já":
                print("appended")
                texts.append(text)
                sources.append(source)
                labels.append(label)
                df_urls.append(url)

        print(text)
        print(source)
        print(label)

        time.sleep(3)
        
# Create a new dataframe from the lists
new_data = pd.DataFrame({'Text': texts, 'Source': sources, 'Label': labels, 'URL': df_urls})

# Concatenate the new dataframe with the existing dataframe
df = pd.concat([df, new_data], ignore_index=True)

-------------
https://www.publico.pt/2023/04/07/politica/noticia/recibos-verdes-estado-aumentaram-14-2045319
-------------
<h2>Área de leitor</h2>
<h2>Fale connosco</h2>
<h2>A frase</h2>
<h2>O contexto</h2>
<h2>Os factos</h2>
<h2>Em suma</h2>
<h2 class="section__title">Em destaque</h2>
<h2 class="section__title">Opinião</h2>
a_frase
HI blockquote
"Em sete anos de governação socialista, o número de trabalhadores a recibos verdes aumentou cerca de 14% na Administração Pública"Requerimento do PSD entregue no Parlamento
"Em sete anos de governação socialista, o número de trabalhadores a recibos verdes aumentou cerca de 14% na Administração Pública"Requerimento do PSD entregue no Parlamento
label tries 1

"Em sete anos de governação socialista, o número de trabalhadores a recibos verdes aumentou cerca de 14% na Administração Pública"Requerimento do PSD entregue no Parlamento
None
-------------
https://www.publico.pt/2023/04/07/politica/noticia/recibos-verdes-estado-aumentaram-14-2045319
---

In [123]:
len(failed_label_urls)

1

In [124]:
len(image_urls)

0

In [125]:
len(urls)

203

In [126]:
len(df_urls)

36

In [127]:
for url in df_urls:
    print(url)

https://www.publico.pt/2023/04/07/politica/noticia/recibos-verdes-estado-aumentaram-14-2045319
https://www.publico.pt/2023/04/04/impar/noticia/politica-francesa-marlene-schiappa-posou-nua-lingerie-playboy-2044934
https://www.publico.pt/2021/06/11/economia/noticia/ana-paula-vitorino-ira-salario-maior-pedro-adao-silva-rui-rio-1966168
https://www.publico.pt/2021/09/13/desporto/noticia/treinador-dinamo-kiev-mircea-lucescu-treinador-titulos-ganhos-actividade-1977311
https://www.publico.pt/2022/07/13/azul/noticia/unico-responsavel-incendios-accao-humana-antonio-costa-2013363
https://www.publico.pt/2023/06/01/azul/noticia/grupo-internacional-activistas-esvazia-pneus-carros-suv-chegou-portugal-2051821
https://www.publico.pt/2022/02/04/politica/noticia/candidaturas-vicepresidentes-pe-aprovadas-1994344
https://www.publico.pt/2023/02/10/politica/noticia/contingentes-imigrantes-suspensos-decisao-governo-moedas-fez-parte-2038395
https://www.publico.pt/2021/03/12/politica/noticia/sair-pais-partir-se

In [128]:
len(texts)

36

In [129]:
len(labels)

36

In [130]:
len(sources)

36

In [131]:
df.head()

,Text,Source,Label,URL
0,"""Em sete anos de governação socialista, o núme...",Requerimento do PSD entregue no Parlamento,Verdadeiro,https://www.publico.pt/2023/04/07/politica/not...
1,A política francesa Marlène Schiappa posou nua...,twitter,Falso,https://www.publico.pt/2023/04/04/impar/notici...
2,"O líder do PSD afirmou que o Governo quer que,...",Rui Rio,Verdadeiro,https://www.publico.pt/2021/06/11/economia/not...
3,"[O Dínamo Kiev] Tem um treinador, Mircea Luces...","Jorge Jesus, treinador do Benfica.",Verdadeiro,https://www.publico.pt/2021/09/13/desporto/not...
4,"“Os incêndios só ocorrem se uma mão humana, vo...","António Costa, primeiro-ministro",Falso,https://www.publico.pt/2022/07/13/azul/noticia...


In [132]:
df.head()
df.to_csv("Publico_fact_check_dataset_p2.csv", index=False)

In [133]:
loaded_df = pd.read_csv("Publico_fact_check_dataset_p2.csv")
loaded_df.head()

,Text,Source,Label,URL
0,"""Em sete anos de governação socialista, o núme...",Requerimento do PSD entregue no Parlamento,Verdadeiro,https://www.publico.pt/2023/04/07/politica/not...
1,A política francesa Marlène Schiappa posou nua...,twitter,Falso,https://www.publico.pt/2023/04/04/impar/notici...
2,"O líder do PSD afirmou que o Governo quer que,...",Rui Rio,Verdadeiro,https://www.publico.pt/2021/06/11/economia/not...
3,"[O Dínamo Kiev] Tem um treinador, Mircea Luces...","Jorge Jesus, treinador do Benfica.",Verdadeiro,https://www.publico.pt/2021/09/13/desporto/not...
4,"“Os incêndios só ocorrem se uma mão humana, vo...","António Costa, primeiro-ministro",Falso,https://www.publico.pt/2022/07/13/azul/noticia...


In [134]:
for source in set(loaded_df['Source']):
    print(source)

nan
Assunção Cristas
António Costa, 8 de Novembro 2021
Excerto da carta aberta enviada pela AVS à Liga portuguesa de futebol
Marcelo Rebelo de Sousa, Presidente da República
redes sociais
António Costa
Marta Temido, podcast do PS
Eurico Brilhante Dias, líder parlamentar do PS
António Costa em entrevista ao Jornal de Notícias, Diário de Notícias e TSF
André Ventura
Manuel Heitor
Rui Tavares, referindo-se ao direito de os partidos indicarem um vice-presidente da Assembleia da República (AR)
Rui Rio
comunicado do PSD
António Costa, primeiro-ministro
Requerimento do PSD entregue no Parlamento
João Oliveira, líder parlamentar do PCP
José Sócrates à CNN
facebook
Jorge Jesus, treinador do Benfica.
António Galamba, último governador civil de Lisboa
. José Silvano, secretário-geral do PSD
Carlos Peixoto, vice-presidente da bancada parlamentar do PSD.
, António Costa, primeiro-ministro
twitter


In [135]:
num_rows_with_none = len(loaded_df[loaded_df['Text'] == 'NONE'])
print(num_rows_with_none)

0
